In [23]:
import scipy.io as sio
data = sio.loadmat('../simulation.mat')

RIRs_raw = data['RIRs']


locations1 = data['locations']

import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import math
from datetime import datetime

# Define the dimensions of the target matrix
num_rows = 132300
num_columns = 150

# Reshape the original vector into the target matrix
RIRs_raw = RIRs_raw[:num_rows * num_columns].reshape(num_rows, num_columns)

num_rows = 150
num_columns = 3

locations1 = locations1[:num_rows * num_columns].reshape(num_rows, num_columns)


def calculate_snr(signal, noise):
    """
    Calculate the Signal-to-Noise Ratio (SNR)
    :param signal: The original signal (actual RIR).
    :param noise: The noise (difference between actual and predicted RIR).
    :return: SNR in decibels.
    """
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

#Algorithm 1

start = datetime.now()
# We Can Replace MLPRegressor with an appropriate regression model

# Example data (replace with actual data)
RIRs = np.array(RIRs_raw)  # 20 RIRs, each with 44.1*3k time samples
locations = np.array(locations1)  # Corresponding 3D coordinates
RIRs = np.delete(RIRs, 22, axis=1)
RIRs = np.transpose(RIRs)
locations = np.delete(locations, 22, axis=0)
# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
losses = []

for train_index, test_index in loo.split(RIRs):
    # print(train_index,"\n-----------------------------------")
    # print(test_index,"\n-----------------------------------")

    X_train, X_test = locations[train_index], locations[test_index]
    y_train, y_test = RIRs[train_index], RIRs[test_index]
    
    # print(X_train,"\n-----------------------------------")
    # print(X_test,"\n-----------------------------------")
    # print(y_train,"\n-----------------------------------")
    # print(y_test,"\n-----------------------------------")

    # Train the model
    model = MLPRegressor(hidden_layer_sizes=(50, 120, 10), activation='relu', alpha=0.2, learning_rate_init=0.00001, max_iter=10000000)
    model.fit(X_train, y_train)

    # print(model,"\n-----------------------------------")
    # Validate the model
    # Ensure X_test is reshaped correctly if it is a single RIR
    X_test_reshaped = X_test.reshape(1, -1) if X_test.ndim == 1 else X_test
    # print(X_test_reshaped,"\n-----------------------------------")

    predicted_RIR = model.predict(X_test_reshaped)
    
    # Calculate and store the loss
    loss = mean_squared_error(y_test, predicted_RIR)
    losses.append(loss)
    SNR = calculate_snr(y_test, (y_test-predicted_RIR))
    print("SNR now is ", SNR)
    # print(loss,"\n-----------------------------------")

    
# print(losses,"\n-----------------------------------")
# Calculate average loss across all cross-validation iterations
average_loss = np.mean(losses)
print(f"Average Loss for AL1 : {average_loss}")

# Predict the RIR at a new location (x1, y1, z1)
new_location = np.array(locations1[22])
new_location = new_location.reshape(1, -1) if new_location.ndim == 1 else new_location
predicted_RIR = model.predict(new_location)
# print(new_location,"\n-----------------------------------")
# print(predicted_RIR,"\n-----------------------------------")

# The predicted_RIR_at_new_location is the RIR in the time domain at (x1, y1, z1)
noise = RIRs_raw[:, 22]-predicted_RIR
absolute_error1 = np.linalg.norm(noise)
SNR1 = calculate_snr(RIRs_raw[:, 22], noise)
print("The absolute error of AL1 in the predicted signal is =", absolute_error1)
print("The SNR of the predicted signal in AL1 is =", SNR1)

end = datetime.now()

print("Algorithm 1 needed",(end - start)/60, "mins to complete.")

with open('results.txt', 'a') as f:
    f.write(f"The absolute error of AL1 in the predicted signal is = {absolute_error1} \nThe SNR of the predicted signal in AL1 is = {SNR1}")

In [9]:
print(locations1[22])

[2.18886212 0.41587333 0.44788202]


In [18]:
print(X_test_reshaped)

[[0.02686126 1.80651146 1.16031358]]


In [21]:
# Predict the RIR at a new location (x1, y1, z1)
new_location = np.array(locations1[22])
new_location = new_location.reshape(1, -1) if new_location.ndim == 1 else new_location
predicted_RIR = model.predict(new_location)
# print(new_location,"\n-----------------------------------")
# print(predicted_RIR,"\n-----------------------------------")

# The predicted_RIR_at_new_location is the RIR in the time domain at (x1, y1, z1)
noise = RIRs_raw[:,22]-predicted_RIR
absolute_error1 = np.linalg.norm(noise)
SNR1 = calculate_snr(RIRs_raw[22], noise)
print("The absolute error of AL1 in the predicted signal is =", absolute_error1)
print("The SNR of the predicted signal in AL1 is =", SNR1)

end = datetime.now()

print("Algorithm 1 needed",(end - start)/60, "mins to complete.")

with open('results.txt', 'a') as f:
    f.write(f"The absolute error of AL1 in the predicted signal is = {absolute_error1} \nThe SNR of the predicted signal in AL1 is = {SNR1}")

The absolute error of AL1 in the predicted signal is = 1.675648386595957
The SNR of the predicted signal in AL1 is = -262.9364447557923
Algorithm 1 needed 0:00:20.807845 mins to complete.


In [20]:
print(predicted_RIR.shape)

(1, 132300)
